![tracker](https://us-central1-vertex-ai-mlops-369716.cloudfunctions.net/pixel-tracking?path=statmike%2Fvertex-ai-mlops%2Farchitectures%2Ftracking%2Fsetup%2Fpixel&file=pixel-schedule-reporting.ipynb)
<!--- header table --->
<table align="left">
  <td style="text-align: center">
    <a href="https://colab.research.google.com/github/statmike/vertex-ai-mlops/blob/main/architectures/tracking/setup/pixel/pixel-schedule-reporting.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Google Colaboratory logo">
      <br>Run in<br>Colab
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/colab/import/https%3A%2F%2Fraw.githubusercontent.com%2Fstatmike%2Fvertex-ai-mlops%2Fmain%2Farchitectures%2Ftracking%2Fsetup%2Fpixel%2Fpixel-schedule-reporting.ipynb">
      <img width="32px" src="https://lh3.googleusercontent.com/JmcxdQi-qOpctIvWKgPtrzZdJJK-J3sWE1RsfjZNwshCFgE_9fULcNpuXYTilIR2hjwN" alt="Google Cloud Colab Enterprise logo">
      <br>Run in<br>Colab Enterprise
    </a>
  </td>      
  <td style="text-align: center">
    <a href="https://github.com/statmike/vertex-ai-mlops/blob/main/architectures/tracking/setup/pixel/pixel-schedule-reporting.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      <br>View on<br>GitHub
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/statmike/vertex-ai-mlops/main/architectures/tracking/setup/pixel/pixel-schedule-reporting.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo">
      <br>Open in<br>Vertex AI Workbench
    </a>
  </td>
</table>

# Reporting

This workflwo focuses on the ongoing management of new load data.

## Source Tables

In [3]:
%%bigquery
SELECT *
FROM `vertex-ai-mlops-369716.pixel_tracking.pixel-tracking-data`
WHERE DATE(event_timestamp) = '2024-04-01'
LIMIT 5

Query is running:   0%|          |

Downloading:   0%|          |

,event_timestamp,file_path,file_name,client,source
0,2024-04-01 14:22:23.905150+00:00,statmike/vertex-ai-mlops,readme.md,github-camo (6bea6b5d),custom
1,2024-04-01 11:50:42.983237+00:00,statmike/vertex-ai-mlops,readme.md,github-camo (6bea6b5d),custom
2,2024-04-01 13:46:25.655135+00:00,statmike/vertex-ai-mlops,readme.md,github-camo (6bea6b5d),custom
3,2024-04-01 14:02:14.287003+00:00,statmike/vertex-ai-mlops,readme.md,github-camo (6bea6b5d),custom
4,2024-04-01 12:29:40.482654+00:00,statmike/vertex-ai-mlops,readme.md,github-camo (6bea6b5d),custom


In [4]:
%%bigquery
SELECT *
FROM `vertex-ai-mlops-369716.reporting.pixel_daily_loads`
WHERE event_date = '2024-04-01'
LIMIT 5

Query is running:   0%|          |

Downloading:   0%|          |

,event_date,file_path,file_name,loads,data_source,row_type,lower_bound,upper_bound
0,2024-04-01,statmike/vertex-ai-mlops,readme.md,43.0,pixel,actual,NaN,NaN
1,2024-04-01,statmike/vertex-ai-mlops/Dev,Notes - Just Notes.ipynb,1.0,pixel,actual,NaN,NaN
2,2024-04-01,statmike/vertex-ai-mlops/MLOps,Vertex AI Pipelines - Control.ipynb,1.0,pixel,actual,NaN,NaN
3,2024-04-01,statmike/vertex-ai-mlops/Dev/new,Geospatial Indexing.ipynb,2.0,pixel,actual,NaN,NaN
4,2024-04-01,statmike/vertex-ai-mlops/Dev/new,Autoencoders.ipynb,1.0,pixel,actual,NaN,NaN


## Update Query

Remove the records for yesterday and today.  Add the updated records for yesterday and today.

**consider** a single Merge statement to combine these.

In [9]:
%%bigquery
DELETE
  FROM `vertex-ai-mlops-369716.reporting.pixel_daily_loads`
  WHERE
    event_date >= DATE_SUB(CURRENT_DATE(), INTERVAL 1 DAY)
    AND
    data_source = 'pixel' and row_type = 'actual'
;
INSERT INTO `vertex-ai-mlops-369716.reporting.pixel_daily_loads`
  WITH DAILY AS (
    SELECT
      EXTRACT(DATE FROM event_timestamp AT TIME ZONE '-05') AS event_date,
      file_path, file_name, COUNT(*) AS loads
    FROM `vertex-ai-mlops-369716.pixel_tracking.pixel-tracking-data`
    WHERE DATE(event_timestamp) >= DATE_SUB(CURRENT_DATE(), INTERVAL 1 DAY)
    GROUP BY event_date, file_path, file_name
  )
  SELECT *,
    'pixel' AS data_source,
    'actual' AS row_type,
    NULL AS lower_bound,
    NULL AS upper_bound
  FROM DAILY
;

Query is running:   0%|          |

""


## Schedule Query

Set this query to run throughout the day, every 2 hours.  Name it `UPDATE_pixel_daily_loads`